# Main Draft 

In [32]:
# ----------------------------------------------------------------------------------------------
#Julia Packages
using Plots
using Pandas
using PyCall
np = pyimport("numpy")
#Question: Add in the Time package?
# ----------------------------------------------------------------------------------------------
# METIS Modules
include("Absopac.jl")
include("Constants.jl")
include("Integrate_transits.jl")
include("Setup_Atmo.jl")


atm_setup_1d (generic function with 1 method)

In [33]:
function gasOpacFunc(Z,T,P, wlrange)
    """
     expects Z in units of Z solar
     expects T in Kelvin, P in pascals
     expects T and P to be 1d arrays of equal length
     expects wlrange to be a 1d array of wavelengths in microns
     
     returns the inverse mean free path in units 1/m
    """
    mu = absopac.mu_func(Z,T,P) # use a varying mean molecular weight throughout the atmosphere
    rho = setup_atm.eos_idealgas(P, mu*gmol_to_kg, T) # use the ideal gas equation of state
    gas = absopac.gasopac_func(Z, wlrange, T, rho)
    rayleigh = absopac.rayleigh_func(Z, wlrange, T, rho)
    return gas + rayleigh # units are light blocked per m, has shape (nT, nwl) or (nwl,) if nz=1
    end 

nalt  = 60 # number of altitude grid layers
nlong = 60 # number of longitude grid points
nlat  = 60 # number of latitude grid points

wlrange = np.logspace(np.log10(0.5),np.log10(5.0),1000) # wavelength array for output spectrum, in microns
nwl = length(wlrange) 

P0 = 10^4.15                # reference pressure in Pascals
Z  = 1.0                     # metallicity in multiples of Z_sun
Mp = 0.48*Mj       # mass of the planet in kg ##Note:pulling from constants?
R0 = 1.2*Rj        # reference radius for the planet where all wavelengths are opaque, in meters
Rstar = 1.05*Rsun  # radius of host star in meters 
Temp = 1350 



1350

In [34]:
# set up the atmosphere as a set of temperatures and pressures  
# on a 3d grid defined by altitude, longitude, latitude
solved_atm = atm_setup_1d(Temp,Mp,R0,P0,nwl,nalt,nlong,nlat, mu_func(Z, T, P0),Z)

# unpack the associated quantities that setup_atm.atm_setup_1d solved for into separate variables
Temps, Pressures, alt_range, long_range, lat_range, atmosphere_grid = solved_atm

# feed those structural quantities into the transit integration routine
clear1d_tspec = effective_area_at_full_depth_no_long_dep(alt_range, long_range, 
            lat_range, Temps, Pressures, gasOpacFunc, nwl, 100, Z, wlrange)/(np.pi*Rstar^2.0) 

LoadError: MethodError: no method matching abs(::typeof(T))
[0mClosest candidates are:
[0m  abs([91m::Union{Float16, Float32, Float64}[39m) at float.jl:486
[0m  abs([91m::Complex[39m) at complex.jl:277
[0m  abs([91m::T[39m) where T<:Measures.Length at ~/.julia/packages/Measures/PKOxJ/src/length.jl:36
[0m  ...

In [31]:
# plot up your results 
plot!(size=(15,5))
plot(wlrange,clear1d_tspec*100,linestyle="--", xlims=(0.5, 5.0), ylims=(1.3,1.6))
xlabel!("Wavelength ($\mu$m)",fontsize=18)
ylabel!("Transit Depth (%)",fontsize=18)
#plt.axis([0.5,5,1.3,1.6])

LoadError: UndefVarError: clear1d_tspec not defined